In [1]:
import numpy as np
import matplotlib.pyplot as plt
import texttable as tt

In [2]:
with open('spectX.txt','r') as fd:
    lines=fd.readlines()
data=np.array([])
temp=np.array([])
for i,line in enumerate(lines):
    if (i!=266):
        temp=np.array(line.split(' ')[0:-1])
    else:
        temp=np.array(line.split(' ')[0:])
    temp=np.reshape(temp,(-1,23))
    if i==0:
        data=np.array(temp)
    else:
         data=np.append(data,temp,axis=0)
data[266,22]='1'
spx=data.astype(int)


In [3]:
with open('spectY.txt','r') as fd:
        lines=fd.readlines()
data=np.array([])
temp=np.empty(shape=(1,1))
for i,line in enumerate(lines):
    data=np.append(data,line[0])
spy=data.astype(int)
spy=np.reshape(spy,(267,1))

In [4]:
def noisy_or(x,p):
    return 1-np.reshape(np.prod(np.power(1-p,x),axis=1),(267,1))

In [5]:
def e_step(x,p,y):
    return (y*x)*p/noisy_or(x,p)

In [6]:
def mistakes(x,p,y):
    ni=noisy_or(x,p)
    mssum=[1 if l!=np.round(m) else 0 for l,m in zip(y,ni)]
    return np.sum(mssum)

In [7]:
def loglikelihood(x,p,y,T):
    llhood=0
    nor=noisy_or(x,p)
    for yi,ni in zip(y,nor):
        if(yi==1):
            llhood+=np.log(ni)
        else:
            llhood+=np.log(1-ni)
    return llhood/T

In [8]:
def p_update(x,y,p,T):
    llhood=[]
    ms=[]
    pold=p
    Ti=np.sum(x,axis=0)
    llhood.append(loglikelihood(x,pold,y,T)) 
    ms.append(mistakes(x,pold,y))
    for i in range(0,256):
        pnew=np.zeros((1,23))
        pnew=np.reshape(np.sum(e_step(spx,pold,spy),axis=0)/Ti,(1,23))
        llhood.append(loglikelihood(x,pnew,y,T))
        ms.append(mistakes(x,pnew,y))
        pold=pnew
    return pnew,llhood,ms
    

In [9]:
p=0.05*np.ones((1,23))
p,llhood,ms=p_update(spx,spy,p,np.shape(spx)[0])

In [12]:

tab=tt.Texttable()
tab.header(['iteration','number of mistakes M','log-likelihood L'])
iteration=[0 if x==-1 else 2**x for x in range(-1,9)]
for i in iteration:
        row=[str(i),str(ms[i]),str(np.round(llhood[i],5))]
        tab.add_row(row)
tab.set_cols_align(['c','c','c'])
s=tab.draw()
print(s)

+-----------+----------------------+------------------+
| iteration | number of mistakes M | log-likelihood L |
+===========+======================+==================+
|     0     |         175          |    [-0.95809]    |
+-----------+----------------------+------------------+
|     1     |          56          |    [-0.49592]    |
+-----------+----------------------+------------------+
|     2     |          43          |    [-0.40822]    |
+-----------+----------------------+------------------+
|     4     |          42          |    [-0.36461]    |
+-----------+----------------------+------------------+
|     8     |          44          |    [-0.3475]     |
+-----------+----------------------+------------------+
|    16     |          40          |    [-0.33462]    |
+-----------+----------------------+------------------+
|    32     |          37          |    [-0.32258]    |
+-----------+----------------------+------------------+
|    64     |          37          |    [-0.3148